**Modélisation du slicing dans les réseaux 5G**
-

**Bárbara Barsi Duarte Batista da Silva**

**Rafaela de Carvalho Machado Pinheiro**


--- 

## Introduction
La 5G permet de déployer des réseaux virtuels de bout en bout, avec des profils de qualité de service (QoS) spécifiques, au-dessus d’une infrastructure physique commune. Le slicing est le terme utilisé pour désigner la fonctionnalité qui rend cette coexistence possible. Chaque réseau virtuel déployé est appelé une tranche dans la terminologie de la 5G. Le découpage en tranches est particulièrement difficile à dimensionner pour respecter les contraintes de qualité de service des tranches. Par exemple, des tranches telles que le haut débit mobile amélioré (eMBB) et les communications ultra fiables à faible latence (URLLC) ont des exigences contradictoires en matière de qualité de service.  
On s’intéresse ici à deux types distincts : les flux URLLC et eMMB. Les premiers sont des flux qui doivent être ultra-fiables et avec une faible latence. Traduit en langage de files d’attente, cela signifie que la perte doit être infime et qu’on ne peut pas se permettre de les retarder par une mise en buffer.  
Pour les flux, eMMB, comme d’habitude, ils passent quand ils peuvent même si nor malement, ils ne devraient pas souffrir de trop de délai.  
Toute la difficulté est de rtouver un moyen physique qui permette de réaliser cette priorisation tout en étant capable d’en étudier les performances pour dimensionner les ressources.  
Dans un premier temps, nous regardons un modèle théorique qui s’étudie relativement bien. Dans un deuxième temps, nous envisagerons un modèle plus réaliste à mettre en oeuvre mais qui s’étudie plus difficilement.

--- 

## 2. Préliminaires

On rappelle que la formule d’Erlang-B donne la probabilité que \( S \) serveurs soient occupés à l’état stationnaire :

$$
ErlB[\rho, S] = \frac{\rho^S}{S! \sum_{j=0}^{S} \frac{\rho^j}{j!}}
$$

On a la relation de récurrence :

$$
\frac{1}{ErlB[\rho, 0]} = 1
$$

$$
\frac{1}{ErlB[\rho, S]} = 1 + \frac{S}{\rho\cdot ErlB[\rho, S-1]}
$$

### Partie 1
1. **Écrire une fonction Python qui renvoie le nombre moyen de clients dans une file M/M/S/S à l’état stationnaire sans calculer de factorielle.**

> Para realizarmos esse cálculo, podemos realizar o cálculo de $\mathbb{E[N]}$, sendo N o número médio de clientes. Como a fórmula da esperança pode ser descrita da forma
>
> $$ \mathbb{E}[N] = \sum_{n \in \mathbb{N}} n \cdot \mathbb{P}(N = n) $$
> 
> e a probabilidade de termos N clientes é no sistema é a mesma de termos N servidores ocupados (já que não há fila de espera), é possível afirmar que o valor de $\mathbb{P}(N = n)$ no estado estacionário se assemelha a $ErlB[\rho, n]$. 
>
> Como a inversa de Erlang-B pode ser descrita por uma relação de recorrência, podemos manipular tais equações a fim de obter uma relação de recorrência para $ErlB[\rho, N]$. Efetuando as manipulações, temos:
>
> $$ ErlB[\rho, 0] = 1 $$
> 
> $$ ErlB[\rho, N] = \frac{1}{1 + \frac{N}{\rho\cdot ErlB[\rho, N-1]}} $$
>
> 
> Com isso, o cálculo do somatório da esperança pode ser calculado utilizando recorrência em $ErlB[\rho, N]$.
> 
> $$ \mathbb{E}[N] = \sum_{n \in \mathbb{N}} n \cdot ErlB[\rho, n]$$

In [17]:
def erlang_b(p, nb_of_servers):

    if (nb_of_servers == 0): return 1 

    return 1/(1 + (nb_of_servers / (p * erlang_b(p, nb_of_servers - 1))))

def mean_number_waiting_customers(arrival_rate, service_rate, nb_of_servers):

    p = arrival_rate / service_rate

    if (nb_of_servers == 1): return 1/(1 + (nb_of_servers/p))

    mean = (nb_of_servers * erlang_b(p, nb_of_servers)) + mean_number_waiting_customers(arrival_rate, service_rate, nb_of_servers - 1)

    return mean

In [18]:
S = 4
lamb = 6
mu = 12

rho = lamb / mu

print(f"{rho = }, {S = }")
print(f"Erl_B = {erlang_b(rho, S):.4f}")
print(f"mean clients = {mean_number_waiting_customers(lamb, mu, S):.4f}")

rho = 0.5, S = 4
Erl_B = 0.0016
mean clients = 0.5315


2. **Pour un choix de paramètre $\rho$ et $S$ tels que $ErlB[\rho, S]$ soit petit (de l’ordre de $10^{-3}$), qu’est-ce que l’on remarque à propos du nombre moyen de clients ?**  
Expliquer ce phénomène en vous aidant des résultats connus sur la M/M/$\infty$.

> &nbsp;
> 
> Considerando que o $ErlB[\rho, S]$ tenha um valor consideravelmente baixo, podemos supor que o sistema tem uma média $\rho$ de clientes em seu estado estacionário. Podemos afirmar isso com base na similaridade do comportamento desse sistema com o que vemos no caso de uma modelagem M/M/$\infty$. Com um valor baixo de Erlang-B, sabemos que o sistema consegue atender os clientes com uma probabilidade significantemente baixa de nenhum dos $S$ servidores estarem disponíveis, enquanto no caso de infinitos servidores, a probabilidade de bloqueio de serviço é nula justamente pela a infinidade de servidores nas quais os clientes podem ser servidos.
> 
> Como temos uma baixa chance de não existirem servidores livres para atendimento, o número médio de clientes no sistema passa a ser guiado pelo tempo de chega e de serviço (ou seja, $\lambda/\mu  = \rho$), e não mais pelo eventual bloqueio de um sistema esgotado. 
> 
> &nbsp;

--- 

## 3. Modélisation

On suit le modèle proposé dans [1] qui n’est pas implémentable dans un système réel mais qui s’analyse mathématiquement très bien.

On considère une file d’attente avec un buffer infini et $ S $ serveurs. Il y a deux classes de clients de type 1 et de type 2. Les clients de type 1 ont une priorité plus élevée que les clients de type 2. Les clients de type 1 arrivent selon un processus de Poisson de paramètre $ \lambda_1 $ et les clients de type 2 arrivent selon un processus de Poisson de paramètre $ \lambda_2 $. Les clients de type 1 ont une durée de service exponentielle de paramètre $ \mu_1 $ et les clients de type 2 ont une durée de service exponentielle de paramètre $ \mu_2 $.

Les clients de type 1 ne peuvent pas être bufferisés et doivent être servis immédiatement. Les clients de type 2 peuvent être bufferisés. On suppose que la capacité de la file d’attente est infinie.

Les clients de classe 1 préemptent les serveurs : s’il reste des serveurs libres, ils s’y mettent normalement mais si tous les serveurs sont pris, ils prennent la place d’un client de classe 2. Celui-ci se retrouve dans le buffer et reprendra son service, là où il en était, dès qu’un serveur se libérera. S’il n’y a que des clients de classe 1 en service, le client de type 1 qui arrive est perdu.

Les clients de classe 2 ne peuvent accéder à un serveur que s’il y en a de libre. S’ils arrivent et qu’il n’y a pas de serveur libre, ils sont mis dans le buffer.

On note $ Q_1 $ le nombre de clients de type 1 dans le système et $ Q_2 $ le nombre de clients de type 2 dans le système. On note $ S_1 $ le nombre de serveurs occupés par des clients de type 1 et $ S_2 $ le nombre de serveurs occupés par des clients de type 2. On note $ B $ le nombre de clients de type 2 dans le buffer.


### Partie 2

1. **Quelles sont les contraintes sur les variables d’état du système et comment les variables $ Q_2 $, $ S_2 $ et $ B $ sont-elles reliées ?**  
   Expliquer en particulier pourquoi $ Q_1 + S_2 < S $ ne peut se produire que si $ B = 0 $.

> &nbsp;
> 
> $Q_2$ is the number of clients of class 2; $S_2$ is the number of servers currently occupied by class 2 clients; $B$ is the occupied buffer, which can only be occupied by class 2 clients, thus it is the number of class 2 clients waiting to be served.  
> From these definitions, we get the following relations:  
> - $S_1 = Q_1$: Type 1 clients do not stay in the buffer, thus every type 1 client in the system is being served.
> - **$S_2 \leq Q_2$**: Since $Q_2 $ is the total number of clients in system, it includes clients in the buffer and clients being served $S_2$. Thus, the number of class 2 clients being served $S_2$ can never exceed the total amount of class 2 clients in the system $Q_2$
> - **$B \leq Q_2$**: Similarly, the number of class 2 clients waiting in the buffer can never be larger than the total amount of class 2 clients in the whole system, because they are the only ones sent to the buffer - as class 1 clients do not wait.
> - **$Q_2 = S_2 + B$**: Only class 2 clients stay in the buffer, so they can eiter be in the buffer or being served.  
> 
> With that, $Q_1 + S_2 < S$ implies $B=0$: The buffer will only be empty ($B=0$) if all class 2 clients are being served. Since class 1 clients cannot wait and are served immediately, we have $Q_1 = S_1$, thus we can rewrite the statement as $S_1 + S_2 < S$. This means there are unoccupied servers, thus there would not be any clients in the buffer.
> 
> &nbsp;


2. **Montrer que le processus $ Q_1 $ est un processus de Markov et reconnaître sa dynamique comme celle d’une file simple dont on précisera les caractéristiques.**

> &nbsp;
> 
> According to the statement, the arrival of class 1 clients follows a Poisson process with parameter $\lambda_1$ and the duration of their services is exponential with parameter $\mu_1$. Type 1 clients do not wait in the buffer and are served right away as they can preempt servers occupied by type 2 clients. Therefore, all $S$ servers are available to type 1 clients.  
> We can model this problem as a Markov process of queue $M/M/S/S$ with the following transition matrix:
> $$
\mathbf{Q} = \begin{pmatrix}
-\lambda_1 & \lambda_1 & 0 & \cdots & 0 & 0 \\
\mu_1 & -(\lambda_1 + \mu_1) & \lambda_1 & \cdots & 0 & 0 \\
0 & 2\mu_1 & -(\lambda_1 + 2\mu_1) & \ddots & 0 & 0 \\
\vdots & \vdots & \ddots & \ddots & \lambda_1 & 0 \\
0 & 0 & \cdots & (S-1)\mu_1 & -[\lambda_1 + (S-1)\mu_1] & \lambda_1 \\
0 & 0 & \cdots & 0 & S\mu_1 & -S\mu_1
\end{pmatrix}
> $$
> &nbsp;


> Podemos modelar o processo Q1 como um sistema M/M/S/S, considerando uma taxa de chegada exponencial de parâmetro $\lambda_1$ e uma taxa de serviço também exponencial de parâmetro $\mu_1$. A parte de caracterização do sistema como S/S se dá devido a impossibilidade dos clientes do tipo 1 se abrigarem no buffer, mas poderem impedir o atendimento de um cliente do tipo 2 para iniciarem os seus próprios (o que garante que todos os $S$ servidores estão a disponibilidade de um cliente do tipo 1 caso estejam livres).
> Dessa forma, temos que a transição de estados pode ser descrita pela matriz a seguir:  
> $$
\mathbf{Q} = \begin{pmatrix}
-\lambda_1 & \lambda_1 & 0 & \cdots & 0 & 0 \\
\mu_1 & -(\lambda_1 + \mu_1) & \lambda_1 & \cdots & 0 & 0 \\
0 & 2\mu_1 & -(\lambda_1 + 2\mu_1) & \ddots & 0 & 0 \\
\vdots & \vdots & \ddots & \ddots & \lambda_1 & 0 \\
0 & 0 & \cdots & (S-1)\mu_1 & -[\lambda_1 + (S-1)\mu_1] & \lambda_1 \\
0 & 0 & \cdots & 0 & S\mu_1 & -S\mu_1
\end{pmatrix}
$$
> &nbsp;
> _**Obs: melhorar resposta, teoricamente ainda está meio fraca (um pouco amparada em intuição e pouca prova matemática)**_




3. **Écrire les transitions possibles du processus de Markov ($Q_1$, $Q_2$).**  
   Montrer en particulier que le taux de transition de l’état $ (q_1, q_2) $ à l’état $ (q_1, q_2-1) $ est donné par :
   $$\min\{q_2, S - q_1\} \mu_2$$


> &nbsp;
> 
> There are 4 possible transitions:
> - $(q_1, q_2) \rightarrow (q_1+1, q_2)$, when a class 1 client arrives with rate $\lambda_1$
> - $(q_1, q_2) \rightarrow (q_1, q_2+1)$, when a class 2 client arrives with rate $\lambda_2$
> - $(q_1, q_2) \rightarrow (q_1-1, q_2)$, when a class 1 client leaves with rate $q_1\mu_1$
> - $(q_1, q_2) \rightarrow (q_1, q_2-1)$, when a class 2 client leaves with rate $min\{q_1, S - q_1\}\mu_2$
> 
> Since type 1 clients can preemt servers serving type 2 clients at any time, we have $S-Q_1$ as the amount of available servers + the amount of servers occupied by type 2 clients, then at most $S-Q_1$ $(Q_2 < S-Q_1)$. This can be rewritten as $Q_1 + Q_2 < S$, and as we proved above, $Q_1 + Q_2 < S \rightarrow B = 0$, which means there are no clients in the buffer, they are all being served ($Q_2 = S_2$). On the other hand, if $B>0$ - meaning there are no free servers - we get that $S_2=S-Q_1$. Thus, $S_2$ is at least equal to $Q_2$ but at most equal to $S-Q_1$, meaning that $S_2=min\{q_1, S - q_1\}\mu_2$  
>
> &nbsp; 

   
4. **Simuler l’évolution de ce système en Python.**  
   On prendra comme valeurs $ S = 10, \mu_1 = 2, \mu_2 = 1, \lambda_1 = 4, \lambda_2 = 3 $.

   On note $ (x_1, x_2) $ le processus ainsi construit. On vérifiera notamment que :

$$
\frac{1}{T} \int_0^T 1_{\{S\}}(x_1(s)) \, ds \xrightarrow{T\rightarrow \infty} \frac{\rho_1^S/S!}{\sum_{j=0}^{S} {\rho_1^j}/{j!}}
$$

In [19]:
def erlang_b(arrival_rate, service_rate, nb_of_servers):

    if (nb_of_servers == 0): return 1 
    p = arrival_rate / service_rate

    return 1/(1 + (nb_of_servers / (p * erlang_b(arrival_rate, service_rate, nb_of_servers - 1))))

In [20]:
import numpy as np

def simple_simulation(S, mu, lambda_, T):

    # Tempo atual e tempo acumulado com todos servidores ocupados
    current_time = 0
    busy_time = 0
    
    # Tempos de liberação de cada servidor (inicialmente todos livres)
    servers = np.zeros(S)
    
    # Gerar tempos entre chegadas e tempos de serviço
    inter_arrivals = np.random.exponential(1/lambda_, size=int(2*lambda_*T))
    arrival_times = np.cumsum(inter_arrivals)
    service_times = np.random.exponential(1/mu, size=len(inter_arrivals))
    
    # Processar cada chegada
    for arrival, service in zip(arrival_times, service_times):
        if arrival > T:
            break  # Termina a simulação
        
        # Atualizar tempo com todos servidores ocupados
        if np.all(servers > current_time):
            busy_time += min(arrival, min(servers)) - current_time
        
        current_time = arrival
        
        # Liberar servidores que terminaram
        servers = np.where(servers <= current_time, 0, servers)
        
        # Verificar se há servidor disponível
        free_server = np.argmin(servers)
        
        if servers[free_server] <= current_time:
            # Atribuir ao servidor livre
            servers[free_server] = current_time + service
        else:
            # Todos servidores ocupados - cliente bloqueado
            pass
    
    # Atualizar tempo final com todos servidores ocupados
    if np.all(servers > current_time) and current_time < T:
        busy_time += min(T, min(servers)) - current_time
    
    # Calcular porcentagem
    percentage_busy = (busy_time / T) * 100
    
    return percentage_busy



percent_busy = simple_simulation(S, mu, lambda_, T)
print(f"Porcentagem do tempo com todos servidores ocupados: {percent_busy:.2f}%")
print(f"Porcentagem do tempo com todos servidores ocupados (Erlang B): {erlang_b(lambda_, mu, S)*100:.2f}%")

NameError: name 'lambda_' is not defined

---

In [ ]:
import numpy as np

def simulation(S, mu1, mu2, lambda1, lambda2, T):
    current_time = 0
    busy_time_class1_full = 0

    q1 = 0  # class 1 in service
    s2 = 0  # class 2 in service
    b2 = 0  # class 2 in buffer

    # Scheduled events: (time, type) -> type in ["arrival1", "arrival2", "dep1_i", "dep2_i"]
    events = []

    # Generate arrival times
    arrival_times1 = np.cumsum(np.random.exponential(1 / lambda1, int(2 * lambda1 * T)))
    arrival_times2 = np.cumsum(np.random.exponential(1 / lambda2, int(2 * lambda2 * T)))
    i1 = i2 = 0

    # Service times for class 1 and class 2
    def gen_service1(): return np.random.exponential(1 / mu1)
    def gen_service2(): return np.random.exponential(1 / mu2)

    servers = []  # list of (end_time, class), class = 1 or 2

    while current_time < T:
        # Schedule next arrivals if available
        next_arrival1 = arrival_times1[i1] if i1 < len(arrival_times1) else float('inf')
        next_arrival2 = arrival_times2[i2] if i2 < len(arrival_times2) else float('inf')

        # Find next departure
        if servers:
            next_departure_time, _ = min(servers, key=lambda x: x[0])
        else:
            next_departure_time = float('inf')

        # Choose the next event
        next_event_time = min(next_arrival1, next_arrival2, next_departure_time)
        if next_event_time > T:
            break

        # Track busy time for class 1 fully occupying servers
        if q1 == S:
            busy_time_class1_full += next_event_time - current_time

        current_time = next_event_time

        if next_event_time == next_arrival1:
            i1 += 1
            if q1 < S:
                q1 += 1
                servers.append((current_time + gen_service1(), 1))
            elif s2 > 0:
                # Preempt a class 2 client
                s2 -= 1
                b2 += 1
                servers = [(t, c) for t, c in servers if not (c == 2 and t == max(t for t, c in servers if c == 2))]
                q1 += 1
                servers.append((current_time + gen_service1(), 1))
            # else: lost

        elif next_event_time == next_arrival2:
            i2 += 1
            if q1 + s2 < S:
                s2 += 1
                servers.append((current_time + gen_service2(), 2))
            else:
                b2 += 1

        elif next_event_time == next_departure_time:
            # Remove the departing customer
            for idx, (t, c) in enumerate(servers):
                if t == next_departure_time:
                    departing_class = c
                    servers.pop(idx)
                    break
            if departing_class == 1:
                q1 -= 1
            else:
                s2 -= 1
                if b2 > 0:
                    b2 -= 1
                    s2 += 1
                    servers.append((current_time + gen_service2(), 2))

    return busy_time_class1_full / T


S = 10              # servers
mu_1 = 2            # service rate
mu_2 = 1
lambda_1 = 4        # arrival rate
lambda_2 = 3
T = 100000           # total simulation time

sim_erlang_b = simulation(S, mu_1, mu_2, lambda_1, lambda_2, T=100000)
theoretical_erlang_b = erlang_b(lambda_1, mu_1, S)

print(f"Difference: {np.abs(sim_erlang_b - theoretical_erlang_b):.4f}")

Difference: 0.0000


## 4. Stationnarité

Il est montré dans [1] que ce système admet un régime stationnaire si et seulement si :

$$
\rho_2 + \frac{1}{\sum_{j=0}^{S} \rho_1^j/j!} \sum_{k=0}^{S} k\frac{\rho_1^k}{k!} < S
$$

où $ \rho_i = \frac{\lambda_i}{\mu_i} $.

### Partie 3

1. **Illustrer ce résultat par simulation. Serait-il possible de deviner (2) par simulation ?**


> &nbsp;
>
> We can show this condition through simulation, but discovering it via simulation is very difficult. Even by simulating a wide range of parameters ($\lambda_1, \lambda_2, \mu_1, \mu_2, S$), we would only get a numerical approximation of the stability boundary, not the exact formula.
> 
> &nbsp;

---

## 5. Calcul de la probabilité stationnaire

En s’aidant de la section 6 de [1], on veut calculer la probabilité stationnaire $ \pi $ de notre système sous réserve que (2) soit satisfaite. Par définition, $ \pi $ est un vecteur de taille infinie indexée par les valeurs possibles de $ (q_1, q_2) $. On numérote les états $ (q_1, q_2) $ en ordre lexicographique à droite :

$ (0, 0) \prec (1, 0) \prec \dots \prec (S, 0) \prec (0, 1) \prec \dots \prec (S, 1) \prec \dots $

et on forme les vecteurs ligne à $ S + 1 $ coordonnées

$$
x_i = (\pi(0,i), \pi(1,i), \dots, \pi(S,i))
$$

On note $ M $ la matrice $ (S+1) \times (S+1) $ qui correspond au générateur d’une file M/M/S/S de paramètres $ \lambda_1 $ et $ \mu_1 $. Le générateur de $ (Q_1, Q_2) $ s’écrit sous forme tri-diagonale par blocs de la manière suivante :

$$
\begin{pmatrix}
M - \lambda_2 Id & \lambda_2 Id \\
A_1 & B_1 & \lambda_2 Id \\
& A_2 & B_2 & \lambda_2 Id \\
\vdots & \vdots & \ddots & \ddots & \ddots & \vdots & \vdots & \vdots & \vdots\\
& & & A_S & B_S & \lambda_2 Id \\
& & & & A_S & B_S & \lambda_2 Id \\
\vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \ddots & \ddots & \vdots\\
\end{pmatrix}
$$

où

$$
B_j = M - A_j - \lambda_2 Id
$$

et

$$
A_j = \text{diag}(a_{n,0}, \dots, a_{n,S})
$$

avec

$$
a_{n,j} = \min(S - j, n) \mu_2.
$$

On admet qu’il existe une matrice $ R_S $ à coefficients positifs (et les plus petits possibles) qui soit solution de l’équation matricielle :

$$
\lambda_2 Id + R_B S + R^2 A_S = 0.
$$

Pour trouver cette matrice $ R $, on réécrit cette équation sous la forme :

$$
R = -(Id + R^2 \tilde{A}_S) \tilde{B}_S^{-1}
$$

où 

$$
\tilde{A}_S=\frac{1}{\lambda_2}A_s \text{ et } \tilde{B}_S=\frac{1}{\lambda_2}B_s
$$

On considère ensuite la suite de matrices

$$
R_0 = 0
$$

$$
R_{n+1} = -(Id + R^2 \tilde{A}_n) \tilde{B}_S^{-1}.
$$

Assez rapidement, cette suite converge vers une matrice solution de l’équation précédente. La propriété selon laquelle cette matrice a les coefficients positifs les plus petits possibles est démontrée dans [2]. On admet également que le rayon spectral de cette matrice est strictement inférieur à 1, donc que $ I - R $ est inversible et que l’on a :

$$
\sum_{j=0}^{\infty} R_j = (Id - R)^{-1}.
$$

### Partie 4

1. **Montrer que la suite $ (x_j, j \geq S) $ définie par**

$$
x_j = x_S R^{j - S} \quad \text{pour} \quad j \geq S.
$$

**est solution des équations d’équilibre au-delà du rang $ S $.**

2. **Établir que**

$$
x_{S-1} = -x_S (\tilde{B}_S + R \tilde{A}_S).
$$

3. **Expliciter par récurrence la suite de matrices $ (T_j, j = S - 1, \dots, 0) $ telle que l’on ait**

$$
x_{j+1} = x_j T_j.
$$

4. **Montrer enfin que l’on a**

$$
x_0 ((M - \lambda_2 I) + T_0 A_1) = 0.
$$

où $ \tilde{M} = \frac{M}{\lambda_2} $.

5. **Expliquer comment on calcule $ x_S $.**

6. **Retrouver la Figure 2 de [1] pour $ S = 5 $.**

--- 

## 6. Canaux de garde

Comme il est difficile d’implémenter la politique préemptive, on se contente souvent d’un système de canaux de garde. Les arrivées de classe 1 occupent un serveur tant qu’il y a un de libre et ne peuvent être mis dans la salle d’attente. En d’autres termes, si tous les serveurs sont pris, éventuellement en partie par des clients de classe 2, les clients de classe 1 qui arrivent sont perdus.

Les clients de classe 2 ne peuvent entrer dans les serveurs que s’il y a au moins $ G $ serveurs libres (avec $ G $ à déterminer mais généralement très petit devant $ S $). Cette règle s’applique à leur arrivée ou au moment où un serveur se libère.

### Partie 5

1. **Pourquoi est-ce que le processus $ (Q_1, Q_2) $ défini précédemment n’est plus un processus de Markov représentant ce système ?**

2. **Représenter la dynamique de ce système par un processus de Markov dont on précisera le générateur infinitésimal.**

3. **Sans faire de calculs, est-ce que la condition de stabilité est plus ou moins contraignante sur $ \rho_2 $ que dans le premier modèle ?**
